In [27]:
from datasets import Dataset, Image
import numpy as np
import pandas as pd

In [2]:
ds_csv = pd.read_csv('NLICorpusData_1400.csv')

ds_csv.head()

,Description,Index,Scenario,AgentType,Difficulty,TimeToComplete,Strategy,Challenging,GeneralComments,Age,...,Occupation,ComputerUsage,DominantHand,EnglishAsFirst,ExpWithRobots,ExpWithRCCars,ExpWithFPS,ExpWithRTS,ExpWithRobotComments,InternalUserID
0,Pick up the yellow cube.,1341,Configuration_1_v1.png,human,5.0,0:00:16,Tried to find something that would differ the ...,Moderately challenging at the beginning but it...,NaN,28,...,Engineer,15-20,Right,1.0,3,1.0,5,3,"Yes, I had to build one in one of my classes",915
1,pick up the yellow block,1363,Configuration_1_v1.png,human,5.0,0:00:14,using left and right,describing the instructions from their point o...,NaN,35,...,retail manager,>20,Right,1.0,2,2.0,5,4,no,911
2,Please pick up the yellow block,1387,Configuration_1_v1.png,human,5.0,0:00:18,"Not really, just addressing each case individu...",The most challenging tasks were the ones where...,NaN,37,...,HR Specalist,>20,Right,1.0,1,1.0,6,5,No,913
3,grab the yellow block.,1438,Configuration_1_v1.png,human,5.0,0:00:10,I followed common sense strategies.,Not challenging. Describing different blocks o...,interesting.,25,...,self employed,>20,Left,1.0,2,2.0,3,3,no,903
4,I would ask them to lift the block that's fart...,1460,Configuration_1_v1.png,human,4.0,0:00:32,No.,I was sort of challenging but nothing I couldn...,Thanks!,31,...,Photographer,15-20,Right,1.0,5,3.0,5,5,Yes. The experience was amusing.,901


In [3]:
# only keep the columns we need, description, index, Scenario
ds_csv = ds_csv[['Description', 'Index', 'Scenario']]

# print the first 5 rows
ds_csv.head()


,Description,Index,Scenario
0,Pick up the yellow cube.,1341,Configuration_1_v1.png
1,pick up the yellow block,1363,Configuration_1_v1.png
2,Please pick up the yellow block,1387,Configuration_1_v1.png
3,grab the yellow block.,1438,Configuration_1_v1.png
4,I would ask them to lift the block that's fart...,1460,Configuration_1_v1.png


In [31]:
block_metadata = pd.read_csv('block_metadata.csv')
# iterate over the rows and create a list of dictionaries

data = []
for index, row in ds_csv.iterrows():
    original_scenario = row['Scenario'][:-4]
    original_scenario = 'v' + original_scenario[-1]

    # remove last 6 characters from scenario
    row['Scenario'] = row['Scenario'][:-7]
    img_config = row['Scenario'][14:]
    row['Scenario'] = row['Scenario'] + '.png'
    row['Scenario'] = 'images/' + row['Scenario']

    # find the block metadata for the current image by comparing the img_config
    block = block_metadata['img_config'] == int(img_config)

    if original_scenario == 'v1':
        block = block & block_metadata['v1'] == 1
    elif original_scenario == 'v2':
        block = block & block_metadata['v2'] == 1

    
    # get index of true value
    block_index = block[block == True].index[0]
    answer = block_metadata.iloc[block_index]['block_id']

    data.append({'description': row['Description'], 'index': row['Index'], 'scenario': row['Scenario'], 'answer': answer})

In [32]:
def gen():
    for d in data:
        yield d

ds = Dataset.from_generator(gen)
ds = ds.cast_column("scenario", Image())

Generating train split: 0 examples [00:00, ? examples/s]

In [33]:
ds.push_to_hub('nli_multimodal_ds')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/382 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Swapnil949/nli_multimodal_ds/commit/3e556586d34d9558641738a7965e07a3d9ec862d', commit_message='Upload dataset', commit_description='', oid='3e556586d34d9558641738a7965e07a3d9ec862d', pr_url=None, pr_revision=None, pr_num=None)